In [1]:
import json 
import pdb
import copy
import glob 
import pandas as pd

In [2]:
def display_template(file, idx, t):
    print('==========={} {}========='.format(file, idx))
    print(t['text']) 
    
    for idx, n in enumerate(t['nodes']):
        print(idx, n)
    print(t['constraints'])
    #if 'interval_type' in t: 
    print('Interval type {}'.format(t['interval_type']))
    if 'unique_obj' in t: print('Unique obj {}'.format(t['unique_obj']))
    if 'answer_obj' in t: print('Answer obj {}'.format(t['answer_obj']))
    if 'all_unique_objs' in t: print('All unique obj {}'.format(t['all_unique_objs']))
    if 'ref_constraints' in t: print('Ref Constraint: {}'.format(t['ref_constraints']))
    if 'ref_remark' in t: print(t['ref_remark'])
    if 'remark' in t: print(t['remark'])

In [3]:
def load_file(file):
    print("Loading template {}...".format(file))
    data = json.load(open(file))
    return data 
    
def save_file(data, file):
    new_file = file.replace('_4.json', '_5.json')
    print("Writing new template to {}...".format(new_file))
    json.dump(data, open(new_file, 'w'), indent=4)
    
def save_file2(data, file):
    new_file = file.replace('_5.json', '_5.json')
    print("Writing new template to {}...".format(new_file))
    json.dump(data, open(new_file, 'w'), indent=4)

In [4]:
def count_unique(d):
    count = 0 
    for n in d['nodes']:
        if 'unique' in n['type']:
            count += 1
    return count

In [5]:
def get_unique_obj(d):
    for idx, n in enumerate(d['nodes']):
        if 'unique' in n['type']:
            return (idx,n)

In [6]:
def get_unique_output_obj(d, last_node_only=False):
    for idx, n in enumerate(d['nodes']):
        if last_node_only and idx!= len(d['nodes'])-1:
            continue 
        if 'filter_count' in n['type']:
            return (idx, n)
        if 'filter_exist' in n['type']:
            return (idx, n)
        if n['type'] == 'exist':
            return (idx, n)
        if n['type'] == 'count':
            return (idx, n)
    return None 

In [7]:
def same_relate_obj(idx, nodes):
    return idx>0 and 'same_' in nodes[idx-1]['type'] and 'query_' in nodes[idx+1]['type']

def spatial_relate_obj(node):
    return  node['type'] == 'relate_action_filter_unique'

In [9]:
def get_all_unique_objs(d, multi_objs=False):
    out = []
    for idx, n in enumerate(d['nodes']):
        if 'unique' in n['type']: # and 'relate' not in n['type']:
            if multi_objs:
                if same_relate_obj(idx, d['nodes']) or spatial_relate_obj(n):
                    out.append((idx, n, 'ans'))
                else:
                    out.append((idx, n, None))
            else:
                out.append((idx, n))
    return out 

In [8]:
def is_exist(d):
    return 'exist' in d['nodes'][-1]['type']

In [9]:
node_type_to_attribute = {'query_size': '<Z>', 'query_shape': '<S>', 'query_color': '<C>', 'query_material': '<M>'}
count = 0
for file in glob.glob('*5.json'):
    data = load_file(file)
    new_data = []
    for d_idx, d in enumerate(data):
        #if d['nodes'][-1]['type'] in node_type_to_attribute and d['interval_type']!='none':
        #display_template(file, d_idx, d)
        count += 1 
        #else:
        #    new_data.append(d)
    #save_file2(new_data, file)
print(count)

Loading template compare_integer_5.json...
Loading template comparison_5.json...
Loading template one_hop_5.json...
Loading template same_relate_5.json...
Loading template zero_hop_5.json...
26


In [12]:
filename = []
template_idx = []
text = []
qtypes = []
interval_types = []
all_constraints = set()
for file in glob.glob('*5.json'):
    data = load_file(file)
    for d_idx, d in enumerate(data):
        #display_template(file, d_idx, d)   
        #pdb.set_trace()
        filename.append(file)
        template_idx.append(d_idx)
        text.append("\n".join(d['text']))
        if 'Is there' in d['text'][0]:
            qtype = 'exist'
        elif 'How many' in d['text'][0] or 'What number' in d['text'][0]:
            qtype = 'count'
        elif 'What' in d['text'][0]:
            qtype = 'query'
        elif 'compare_int' in file:
            qtype = 'compare_integer'
        elif 'comparison' in file:
            qtype = 'comparison'
        qtypes.append(qtype)
        interval_types.append(d['interval_type'])
        for ct in d['constraints']:
            all_constraints.add(ct['type'])
        
out = {
    #"filename": filename,
    #"template_idx": template_idx,
    "question_category": qtypes, 
    "interval_category": interval_types,
    "text": text
}    
df = pd.DataFrame.from_dict(out)
df.to_excel('templates_sheet.xlsx')

Loading template compare_integer_5.json...
Loading template comparison_5.json...
Loading template one_hop_5.json...
Loading template same_relate_5.json...
Loading template zero_hop_5.json...


In [13]:
df

,question_category,interval_category,text
0,compare_integer,compositional,Does the <Z> <C> <M> <S> <A> as frequently as ...
1,compare_integer,compositional,Does the <Z> <C> <M> <S> <A> less frequently t...
2,compare_integer,compositional,Does the <Z> <C> <M> <S> <A> more frequently t...
3,comparison,compositional,Do the <Z> <C> <M> <S> and the <Z2> <C2> <M2> ...
4,comparison,compositional,Do the <Z> <C> <M> <S> and the <Z2> <C2> <M2> ...
5,count,atomic,What number of <A2> <Z2> <C2> <M2> <S2>s are <...
6,exist,atomic,Is there any <A2> <Z2> <C2> <M2> <S2> <R> the ...
7,query,atomic,What is the <A2> <Z2> <C2> <M2> <S2> [that is]...
8,exist,compositional,Is there anything else that performs the same ...
9,exist,compositional,Is there anything else that performs the same ...


In [14]:
data = json.load(open('synonyms.json'))
original = []
synonyms = []
for k,vs in data.items():
    original.append(k)
    synonyms.append('\n'.join(vs))
out = {
    'original': original,
    'synonyms': synonyms
}
df = pd.DataFrame.from_dict(out)
df.to_excel('synonyms.xlsx')

In [15]:
df

,original,synonyms
0,thing,thing\nobject
1,sphere,sphere\nball
2,cube,cube\nblock
3,spl,snitch
4,large,large\nbig
5,small,small
6,medium,average
7,metal,metallic\nmetal\nshiny
8,rubber,rubber\nmatte
9,static,stationary
